### Testing Bootstrapping against MCMC

In [1]:
# math, array manipulation, etc.
import numpy as np

# timing
from timeit import default_timer

import astropy.io.fits as fits
from astropy.table import Table                    # Table data structure
import astropy.units as u

# necessary utilities from scipy, astropy and photutils
from scipy.optimize import differential_evolution
from scipy.ndimage import maximum_filter, gaussian_filter
from astropy.modeling import functional_models
from astropy.convolution import convolve, Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma, SigmaClip
from photutils import background
from photutils.background import Background2D, StdBackgroundRMS
from photutils.segmentation import deblend_sources, SegmentationImage, detect_sources

# plots
import matplotlib.pyplot as plt
import corner                       # giant triangle confusograms!

#%matplotlib inline

# MCMC sampling package
import emcee

import os

import sys


In [2]:
sys.path.append('/Users/aidan/Desktop/sl_project/galaxy-galaxy-sel/')

from imfit_mcmc import SersicMCMC

In [82]:
jr_path = '/Users/aidan/Desktop/sl_project/img_cutouts/sl_jacobs/rband_dr2/no_cluster_scale/mask_works/fit_works/'
jg_path = '/Users/aidan/Desktop/sl_project/img_cutouts/sl_jacobs/gband_dr2/no_cluster_scale/mask_works/fit_works/'

files = np.array(os.listdir(jr_path))
finds = [i for i, f in enumerate(files) if 'DESJ' in f]
files_r = files[finds]

files = np.array(os.listdir(jg_path))
finds = [i for i, f in enumerate(files) if 'DESJ' in f]
files_g = files[finds]

index = 5

filename_r = os.path.join(jr_path, files_r[index])
filename_g = os.path.join(jg_path, files_g[index])

img_r = fits.open(filename_r)[0].data
img_g = fits.open(filename_g)[0].data


In [ ]:
# begin by calling a quick fit

# then test bootstrapping using the example in the pyimfit documentation

# then run an MCMC call and compare posterior distributions using corner
